In [1]:
import urllib.parse
import numpy as np
import pandas as pd
import random as rd
from scipy.stats import bernoulli

domain = "51.91.251.0"
port = 3000
host = f"http://{domain}:{port}"
path = lambda x: urllib.parse.urljoin(host, x)

In [2]:
import requests

user_id = '70868297-ced7-46d3-8a7a-4ca528e49009'
name = 'hugo-first-avatar'
r = requests.post(path(f'avatars/{user_id}/{name}'))
r

<Response [409]>

In [3]:
pricing_requests = pd.read_csv("pricing_requests.csv")
test_sample = pd.read_csv("test_set.csv")

In [4]:
list_cities = ["amsterdam", "copenhagen", "madrid", "paris", "rome", "sofia", "valletta", "vienna", "vilnius"]
list_date = [k for k in range(45)]
list_languages = ["austrian", "belgian", "bulgarian", "croatian", "cypriot", "czech", "danish", "dutch",
                  "estonian", "finnish", "french", "german", "greek", "hungarian", "irish", "italian",
                  "latvian", "lithuanian", "luxembourgish", "maltese", "polish", "portuguese", "romanian",
                  "slovakian", "slovene", "spanish", "swedish"]

In [5]:
def GetDatesTest(test_sample):
    list_dates_test = []
    for avatar_id in test_sample.avatar_id.unique():
        dates = [ind for ind in test_sample.loc[test_sample.avatar_id == avatar_id, "date"].unique()]
        list_dates_test.append(dates)
    return(list_dates_test)

def GetCitiesLanguagesTest(test_sample):
    list_cities_languages_test = []
    for rq_nb in test_sample.order_requests.unique():
        temp = test_sample.loc[test_sample.order_requests == rq_nb, ["city","language"]].drop_duplicates()
        list_cities_languages_test.append([temp.city.values[0], temp.language.values[0]])
    return(list_cities_languages_test)

def CreateWiseRequestParams(nb_avatars, list_dates, list_cities_languages, p_mobile):
    list_params = []
    for i in range(nb_avatars):
        num_avatar = rd.randint(0, 100000)
        name = "AvatarNumero"+str(num_avatar)
        dates_avatar = rd.choice(list_dates)
        city_language = rd.choice(list_cities_languages)
        mobile = np.random.binomial(1,p_mobile)
        language = city_language[1]
        city = city_language[0]
        r = requests.post(path(f"avatars/{user_id}/{name}"))
        for date in np.sort(dates_avatar)[::-1]:
            params = {"avatar_name": name,
                       "language": language,
                       "city": city,
                       "date": date,
                       "mobile": mobile,
                    }
            list_params.append(params)
    return(list_params)

list_dates_test = GetDatesTest(test_sample)
list_cities_languages_test = GetCitiesLanguagesTest(test_sample)
temp_mobile = test_sample.loc[:,["mobile","order_requests"]].drop_duplicates()
p_mobile_test = len(temp_mobile[temp_mobile.mobile == 1])/len(temp_mobile)


In [6]:
def CreateRequestParams(nb_avatars, nb_requests):
    list_params = []
    for i in range(nb_avatars):
        num_avatar = rd.randint(0, 100000)
        name = "AvatarNumero"+str(num_avatar)
        if nb_requests > 1 :
            date = np.random.randint(12,44)
        else: date = rd.choice(list_date)
        r = requests.post(path(f"avatars/{user_id}/{name}"))
        for j in range(nb_requests):
            change = np.random.binomial(1,0.7)
            if change or j == 0:
                language = rd.choice(list_languages)
                city = rd.choice(list_cities)
                mobile = rd.randint(0,1)
            params = {"avatar_name": name,
                       "language": language,
                       "city": city,
                       "date": date,
                       "mobile": mobile,
                    }
            date = date - np.random.randint(1,4)
            if params not in list_params:
                list_params.append(params)
    return(list_params)

In [7]:
def AddRequestOrder(df, request_order = 1):
    data = df.copy()
    data["request_order"] = np.nan
    data["avatar_nb_requests"] = np.nan
    avatar_id = data.loc[0,"avatar_id"]
    date = data.loc[0,"date"]
    city = data.loc[0,"city"]
    language = data.loc[0,"language"]
    mobile = data.loc[0,"mobile"]
    avatar_nb_requests = 1
    for k in range(len(data)):
        if data.loc[k,"avatar_id"] != avatar_id :
            avatar_id = data.loc[k,"avatar_id"]
            date = data.loc[k,"date"]
            city = data.loc[k,"city"]
            language = data.loc[k,"language"]
            mobile = data.loc[k,"mobile"]
            avatar_nb_requests = 1
            request_order += 1
        elif data.loc[k,"date"] != date or data.loc[k,"city"] != city or data.loc[k,"language"] != language or data.loc[k,"mobile"] != mobile:
            date = data.loc[k,"date"]
            city = data.loc[k,"city"]
            language = data.loc[k,"language"]
            mobile = data.loc[k,"mobile"]
            avatar_nb_requests += 1
            request_order += 1
        data.loc[k,"request_order"] = request_order
        data.loc[k,"avatar_nb_requests"] = avatar_nb_requests
    return data
        

In [8]:
def MakeRequests(list_params):
    pricing_requests = []
    for params in list_params:
        try:
            req = requests.get(path(f"pricing/{user_id}"), params=params)
            pricing_requests.append(
            pd.DataFrame(req.json()['prices']).assign(**req.json()['request']))
        except KeyError:
            print("vla ti pas une erreur bondiou")
    pricing_requests = pd.concat(pricing_requests, ignore_index = True)
    return(pricing_requests)

def MergeRequestsResults(Big_df, New_df):
    big_df = Big_df.copy()
    request_order = big_df.request_order[len(big_df)-1] + 1
    new_df = AddRequestOrder(New_df, request_order = request_order)
    res = pd.concat([big_df, new_df], ignore_index = True)
    return(res)

In [11]:
new_pricing_requests = pd.read_csv('new_pricing_requests.csv')

In [ ]:
list_params = CreateWiseRequestParams(1, list_dates_test, list_cities_languages_test, p_mobile_test)
New_df = MakeRequests(list_params)
new_pricing_requests = MergeRequestsResults(new_pricing_requests, New_df)

In [13]:
new_pricing_requests.to_csv('new_pricing_requests.csv', index=False)